In [ ]:
## DATA MANIPULATION
import numpy as np
import pickle

from collections import defaultdict

## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

## CLASSIFICATION
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

### Load preprocessed data

In [ ]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)
labels = train.columns[2:]

### TF-IDF weighted Word2Vec vectorizer
- TF-IDF down-weighs frequent terms and up-weighs rare terms, which helps identify indicators for toxic comments (i.e., they're less common)

In [ ]:
## BUILD WORD2VEC EMBEDDINGS DICTIONARY
embeddings_dict = dict()
f = open(r'../data/GoogleNews-vectors-negative300.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    embeddings_dict[word] = vec
f.close()
print('Extracted {} word vectors'.format(len(embeddings_dict)))

In [ ]:
## SKLEARN COMPATIBLE TF-IDF-Word2Vec VECTORIZER TRANSFORMER 
class tfidf_w2c(object):
    def __init__(self, embeddings_dict):
        # Glove Embeddings Dictionary
        self.embeddings_dict = embeddings_dict
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(embeddings_dict[next(iter(embeddings_dict))])
        # Initialize TF-IDF
        self.tfidf = TfidfVectorizer(min_df=3,
                                    token_pattern = '\S+', # preserve each word in preprocessed text
                                    max_df=0.9, 
                                    strip_accents='unicode',
                                    sublinear_tf=1)
        # TF-IDF weights dictionary
        self.weights_dict = None

    def fit(self, X, y):
        '''
        fit tfidf and create weight dictionary
        credit: github.com/nadbordrozd
        '''
        self.tfidf.fit(X)
        # if a word doesn't exist, it needs to be considered as 
        # infrequent as the most infrequent known words, i.e., 
        # it should have the highest known idf value.
        max_idf = max(self.tfidf.idf_)
        self.weights_dict = defaultdict(
            lambda: max_idf, 
            [(w, self.tfidf.idf_[i]) for w, i in self.tfidf.vocabulary_.items()])
        return self
    
    def sentence2vec(self, s):
        '''
        Input:
        Sentence string

        Transformations:
        Get vector for each word -> 
        weigh the vector by idf value ->
        Average vectors

        Output:
        Vector for sentence
        '''
        words = s.split()
        M = []
        for w in words:
            try:
                M.append(self.embeddings_dict[w] * self.weights_dict[w])
            except:
                continue
        M = np.array(M)
        v = M.mean(axis=0)
        if type(v) != np.ndarray:
            return np.zeros(self.dim)
        return v # / np.sqrt((v ** 2).sum())

    def transform(self, X):
        X = X.apply(self.sentence2vec)
        return np.stack(X.values, axis=0)

In [ ]:
vec = tfidf_w2c(embeddings_dict)
X_train = vec.fit(train['comment_text'], train[labels[0]]).transform(train['comment_text'])
X_valid = vec.transform(valid['comment_text'])

### SVM

In [ ]:
## CREATE A MODEL FOR EACH CATEGORY
for label in labels:
    # Create and fit model
    m = LinearSVC(class_weight='balanced')
    m.fit(X_train, train[label].values)
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds),
                                    f1_score(valid[label], preds)))

### Logistic regression

In [ ]:
## CREATE A MODEL FOR EACH CATEGORY
for label in labels:
    # Create & Fit model
    m = LogisticRegression(solver='saga',class_weight='balanced')
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds),
                                    f1_score(valid[label], preds)))

### XGBoost

In [ ]:
## CREATE A MODEL FOR EACH CATEGORY
for label in labels:
    # Create & Fit model
    m = XGBClassifier(n_estimators=100,
                      scale_pos_weight= sum(train[label]==0) / sum(train[label]==1),
                      n_jobs=-1)
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds),
                                    f1_score(valid[label], preds)))